In [ ]:
# default_exp search

# search

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import src.setEnv as _
from nicHelper.dictUtil import genSchema
import yaml
import os
# os.environ['BRANCH'] = 'master'

In [ ]:
#export
from src.getEnv import *
from src.getConfig import getConfig
from sentry_sdk.integrations.aws_lambda import AwsLambdaIntegration
from sentry_sdk import add_breadcrumb, capture_exception
from awsSchema.apigateway import Response, Event, Headers
from beartype import beartype
from diskcache import Cache
from datetime import datetime
import boto3, sentry_sdk, json

github token is ****************************************


## init sentry

In [ ]:
#export
sentry_sdk.init(
    dsn="https://3523e6e8c5e14d33bbc9f64965af96c1@o839457.ingest.sentry.io/5862083",
    integrations=[AwsLambdaIntegration()]
  )

def logSearch(message:str, data:(list,dict)):
  add_breadcrumb(
    category='search',
    data=data,
    level='info',
    message=message
  )
def logValidation(message:str, data:(list,dict)):
  add_breadcrumb(
    category='validation',
    data=data,
    level='info',
    message=message
  )
def logTransform(message:str, data:(list,dict)):
  add_breadcrumb(
    category='transformation',
    data=data,
    level='info',
    message=message
  )
def logConfig(message:str, data:(list,dict)):
  add_breadcrumb(
    category='loadingConfig',
    data=data,
    level='info',
    message=message
  )
def logParseResult(message:str, data:(list,dict)):
  add_breadcrumb(
    category='parseResult',
    data=data,
    level='info',
    message=message
  )

In [ ]:
#export
class ResponseParsingError(Exception):
  pass

class QueryParsingError(Exception):
  pass

class ConfigLoadingError(Exception):
  pass

class SearchError(Exception):
  pass

In [ ]:
#export
cache:Cache = Cache('/tmp/')
  
@beartype
@cache.memoize(tag='search',expire=900)
def search(query:str, config:dict, fuzzyAmount = 0)->dict:
  client = boto3.client('cloudsearchdomain', endpoint_url=SEARCHENDPOINT)
  
  #log
  logSearch('searching at the following config', {'query':query, 'config':config, 'endpoint': SEARCHENDPOINT})
  
  fuzzySuffix = f'~{fuzzyAmount}' if fuzzyAmount else ""
  
  response = client.search(
    query = query + fuzzySuffix,
    **config
    )
  logSearch('searchResponse: ', response)
  return response

In [ ]:
#export
def lambdaSearch(event, *args):
  ## get query
  try:
    logValidation('parsing query', event)
    query = Event.parseQuery(event)['query']
    logValidation('parsing success', {'query':query})
  except KeyError as e:
    capture_exception(QueryParsingError(e))
    return Response.returnError(f'missing or wrong parameters \n{e}')
  except Exception as e:
    capture_exception(QueryParsingError(e))
    return Response.returnError(f'unknown error while parsing queryparam \n {e}')
  
  try:
    logConfig('loading config', {})
    config = getConfig()
    logConfig('config loaded', config)
  except Exception as e:
    capture_exception(ConfigLoadingError(e))
    return Response.returnError(f'unable to get config {e}')
  
  try:
    response = search(query=query, config=config)
  except Exception as e:
    capture_exception(SearchError(e))
    return Response.returnError(f'''
      error while searching, please check if config is correct, 
      dont forget to invalidate after updating config {e}
      ''')
  
  try:
    logParseResult('parsing result', response)
    results = response['hits']['hit']
    logParseResult('result hits', results)
    fields = [result['fields'] for result in results]
    logParseResult('fields', fields)
    filteredFields = [{'cprcode': int(field['cprcode'][0]), 'pr_engname': field['pr_engname'][0] } for field in fields]
    logParseResult('filtered fields', filteredFields)
    return Response.returnSuccess(filteredFields)
  
  except Exception as e:
    capture_exception(ResponseParsingError(e))
    return Response.returnError(f'parsing result error, plz contact developer {datetime.now().timestamp()}')
  




In [ ]:
cache.clear()
%time r = lambdaSearch(Event.getInput(queryStringParameters={'query': 'strawberry'}))
%timeit r = lambdaSearch(Event.getInput(queryStringParameters={'query': 'strawberry'}))
r = lambdaSearch(Event.getInput(queryStringParameters={'query': 'strawberry'}))
if r['statusCode'] >=400:
  print(Response.parseBody(r.copy()))
print(yaml.dump(Response.parseBody(r.copy())[:10]))

CPU times: user 48.9 ms, sys: 10.9 ms, total: 59.8 ms
Wall time: 414 ms
20.5 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
- cprcode: 66134
  pr_engname: WENNEKER STRAWBERRY 70 CL.
- cprcode: 61482
  pr_engname: JELLO STRAWBERRY 85 G.
- cprcode: 146557
  pr_engname: MARKENBURG STRAWBERRT 80 G.
- cprcode: 60225
  pr_engname: STUTE STRAWBERRY JAM 430 G.
- cprcode: 183608
  pr_engname: KOOL-AID STRAWBERRY  3.9 G.
- cprcode: 244281
  pr_engname: GIGGLES STRAWBERRY1-6YEARS
- cprcode: 99337
  pr_engname: DARBO ERDBEERE STRAWBERRY 70%FRUIT 200g
- cprcode: 202378
  pr_engname: REKORDERLIG STRAWBERRY & LIME
- cprcode: 183607
  pr_engname: KOOL-AID STRAWBERRY KIWI 4.8 G
- cprcode: 209696
  pr_engname: VOLA TWIZT(STRAWBERRY+LNCHEE)275ML



In [ ]:
def testSearch(term:str):
  input_ = Event.getInput(queryStringParameters={'query': term})
  r = Response.parseBody(lambdaSearch(input_))
  return yaml.dump(r)

In [ ]:
print(testSearch('woolworth'))

- cprcode: 239840
  pr_engname: WW SELECT UNSWEETENED ALMOND MILK 1L
- cprcode: 239842
  pr_engname: WW FREE FROM GLUTEN GINGERNUT BISCUIT 15
- cprcode: 239847
  pr_engname: WW FREE FROM GLUTEN BREADCRUMBS 350G
- cprcode: 239841
  pr_engname: WW FREE FROM DOUBLE  CHOC BISCUIT 160G
- cprcode: 239844
  pr_engname: WW FREE FROM CREME  CHOC BISCUITS 120G
- cprcode: 239845
  pr_engname: WW FREE FROM GLUTEN CHOCOLATE CHIP BISCU
- cprcode: 239843
  pr_engname: WW FREE FROM MINT  CREME BISCUIT 145G
- cprcode: 235229
  pr_engname: WW SNACKES 150 G.
- cprcode: 235235
  pr_engname: WW SECRET BURGER SAUCE 330 ML.
- cprcode: 235259
  pr_engname: WW SCOTCH FINGER 250 G.
- cprcode: 235231
  pr_engname: WW CROWN MINTS 225 G.
- cprcode: 235252
  pr_engname: WW JUICE LIME 250 ML.
- cprcode: 235250
  pr_engname: WW PARTY MIX 150 G.
- cprcode: 236758
  pr_engname: WW  LEMON JUICE 250 ML.
- cprcode: 235238
  pr_engname: WW SAUCE TOMATO SQUEEZE 500 ML.
- cprcode: 235232
  pr_engname: WW EXTRA STRONG MINTS 2

In [ ]:
getConfig()

{'returnFields': 'cprcode,pr_engname', 'size': 100}

In [ ]:
getConfig()

{'returnFields': 'cprcode,pr_engname', 'size': 100}

In [ ]:
query = 'banana'
config = {'returnFields':'cprcode,pr_engname'}
r = search(query,config)
results = r['hits']['hit']
results
# fields = [result['fields'] for result in results]
# filteredFields = [{'cprcode': int(field['cprcode'][0]), 'pr_engname': field['pr_engname'][0] } for field in fields]
# filteredFields

[{'id': '226238', 'fields': {'pr_engname': ['BANANA'], 'cprcode': ['226238']}},
 {'id': '203812',
  'fields': {'pr_engname': ['BANANA PUFF'], 'cprcode': ['203812']}},
 {'id': '167300',
  'fields': {'pr_engname': ['BANANA RAISIN'], 'cprcode': ['167300']}},
 {'id': '167023',
  'fields': {'pr_engname': ['BANANA CAKE'], 'cprcode': ['167023']}},
 {'id': '223897',
  'fields': {'pr_engname': ['AMBROSIA BANABA CUSTARD POT 150G.'],
   'cprcode': ['223897']}},
 {'id': '132329',
  'fields': {'pr_engname': ['SHAKER BANANA 70 CL.'], 'cprcode': ['132329']}},
 {'id': '102838',
  'fields': {'pr_engname': ['Banana Boat Aloe Vera Gel 453gm'],
   'cprcode': ['102838']}},
 {'id': '232576',
  'fields': {'pr_engname': ['SMOOTHIE BERRY BANANA'], 'cprcode': ['232576']}},
 {'id': '232566',
  'fields': {'pr_engname': ['FRAPPE CHOX BANANA'], 'cprcode': ['232566']}},
 {'id': '102833',
  'fields': {'pr_engname': ['BANANA BOAT SUN PROTECTION SPF50 118 ML'],
   'cprcode': ['102833']}}]